In [2]:
import json
import numpy as np
import os

cw = os.getcwd()
pwd = os.path.abspath(os.path.join(cw, "..", "results"))
print(f"Parent working directory: {pwd}")

Parent working directory: /proj/sourasb-220503/IoT_attack_CL_IDS/results


## WCL

In [24]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_WCL_random.json",
    pwd + "/2_experiment_results_LSTM_WCL_random.json",
    pwd + "/3_experiment_results_LSTM_WCL_random.json"
]

values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.24872249170239052
Number of entries: 48
Average domain training cost: 17.305932822161896
Number of entries: 47
Average FWT value for this file: -0.25726150884333365
Number of entries: 48
Average domain training cost: 15.667352932903063
Number of entries: 47
Average FWT value for this file: -0.2570597378041926
Number of entries: 48
Average domain training cost: 16.740021244598513
Overall average domain training cost: 16.57110233322116±0.6795273908320675
Overall average FWT value: -0.25434791278330554±0.003978626198514741


In [25]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/2_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/3_experiment_results_LSTM_WCL_b2w.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.12268918743474294
Number of entries: 48
Average domain training cost: 16.067146999921533
Number of entries: 47
Average FWT value for this file: -0.1247477123744907
Number of entries: 48
Average domain training cost: 14.832972631847952
Number of entries: 47
Average FWT value for this file: -0.1403944409810955
Number of entries: 48
Average domain training cost: 15.909531362965936
Overall average domain training cost: 15.60321699824514±0.5484328948486981
Overall average FWT value: -0.12927711359677638±0.007905930573921058


In [26]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/2_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/3_experiment_results_LSTM_WCL_w2b.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.4192760750719144
Number of entries: 48
Average domain training cost: 21.653915387568606
Number of entries: 47
Average FWT value for this file: -0.399570268903927
Number of entries: 48
Average domain training cost: 21.713241756238858
Number of entries: 47
Average FWT value for this file: -0.38877166878181973
Number of entries: 48
Average domain training cost: 21.641707597038476
Overall average domain training cost: 21.66962158028198±0.031244171192812312
Overall average FWT value: -0.402539337585887±0.012629099397989247


In [27]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/2_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/3_experiment_results_LSTM_WCL_toggle.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.3082065475963028
Number of entries: 48
Average domain training cost: 16.323437775684095
Number of entries: 47
Average FWT value for this file: -0.31400652969003573
Number of entries: 48
Average domain training cost: 16.65995510198506
Number of entries: 47
Average FWT value for this file: -0.322233304494511
Number of entries: 48
Average domain training cost: 16.35760162078562
Overall average domain training cost: 16.44699816615159±0.15122782915789676
Overall average FWT value: -0.3148154605936165±0.005754896707757308


## LWF

In [28]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_LwF_random_alpha_1.0_T_4.0.json",
    pwd + "/2_experiment_results_LSTM_LwF_random_alpha_1.0_T_4.0.json",
    pwd + "/3_experiment_results_LSTM_LwF_random_alpha_1.0_T_4.0.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.06930268164874667
Number of entries: 48
Average domain training cost: 23.455956973770906
Number of entries: 47
Average FWT value for this file: -0.06497994549033356
Number of entries: 48
Average domain training cost: 24.963494970138225
Number of entries: 47
Average FWT value for this file: -0.06658163506453542
Number of entries: 48
Average domain training cost: 25.274890915476135
Overall average domain training cost: 24.564780953128423±0.7942962594195778
Overall average FWT value: -0.06695475406787188±0.0017843626904866493


In [29]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_LwF_b2w_alpha_1.0_T_4.0.json",
    pwd + "/2_experiment_results_LSTM_LwF_b2w_alpha_1.0_T_4.0.json",
    pwd + "/3_experiment_results_LSTM_LwF_b2w_alpha_1.0_T_4.0.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.09509467494501585
Number of entries: 48
Average domain training cost: 25.465951746242354
Number of entries: 47
Average FWT value for this file: -0.07627483276220344
Number of entries: 48
Average domain training cost: 25.66495840761733
Number of entries: 47
Average FWT value for this file: -0.08083461411067383
Number of entries: 48
Average domain training cost: 25.060380479015294
Overall average domain training cost: 25.397096877624993±0.25157419400966385
Overall average FWT value: -0.08406804060596436±0.00801614621131736


In [31]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_LwF_w2b_alpha_1.0_T_4.0.json",
    pwd + "/2_experiment_results_LSTM_LwF_w2b_alpha_1.0_T_4.0.json",
    pwd + "/3_experiment_results_LSTM_LwF_w2b_alpha_1.0_T_4.0.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.11893662089891402
Number of entries: 48
Average domain training cost: 23.983616102816693
Number of entries: 47
Average FWT value for this file: -0.1533875064553216
Number of entries: 48
Average domain training cost: 23.735634884549654
Number of entries: 47
Average FWT value for this file: -0.13612862992825872
Number of entries: 48
Average domain training cost: 24.919521169824293
Overall average domain training cost: 24.212924052396883±0.5097929553069219
Overall average FWT value: -0.13615091909416477±0.014064523964224328


In [32]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_LwF_toggle_alpha_1.0_T_4.0.json",
    pwd + "/2_experiment_results_LSTM_LwF_toggle_alpha_1.0_T_4.0.json",
    pwd + "/3_experiment_results_LSTM_LwF_toggle_alpha_1.0_T_4.0.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.11958894852840515
Number of entries: 48
Average domain training cost: 24.353824891213055
Number of entries: 47
Average FWT value for this file: -0.12852042444959846
Number of entries: 48
Average domain training cost: 24.656546526525442
Number of entries: 47
Average FWT value for this file: -0.15567008190801954
Number of entries: 48
Average domain training cost: 25.535848347232484
Overall average domain training cost: 24.848739921656996±0.5013306217751405
Overall average FWT value: -0.13459315162867438±0.015343197364152327


## EWC

In [33]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_EWC_random.json",
    pwd + "/2_experiment_results_LSTM_EWC_random.json",
    pwd + "/3_experiment_results_LSTM_EWC_random.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.05046301542075671
Number of entries: 48
Average domain training cost: 102.06821682047546
Number of entries: 47
Average FWT value for this file: -0.11172019307541328
Number of entries: 48
Average domain training cost: 104.72575301183194
Number of entries: 47
Average FWT value for this file: -0.07425225496373478
Number of entries: 48
Average domain training cost: 104.23642242981684
Overall average domain training cost: 103.67679742070807±1.154847685543385
Overall average FWT value: -0.07881182115330158±0.025215110369661547


In [34]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_EWC_b2w.json",
    pwd + "/2_experiment_results_LSTM_EWC_b2w.json",
    pwd + "/3_experiment_results_LSTM_EWC_b2w.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.4192760750719144
Number of entries: 48
Average domain training cost: 21.653915387568606
Number of entries: 47
Average FWT value for this file: -0.08373758071275778
Number of entries: 48
Average domain training cost: 96.30149935296504
Number of entries: 47
Average FWT value for this file: -0.07714951490751562
Number of entries: 48
Average domain training cost: 100.19488818571456
Overall average domain training cost: 72.71676764208273±36.14185735335599
Overall average FWT value: -0.1933877235640626±0.1597498276743247


In [35]:

# List of files
files = [ pwd + "/1_experiment_results_LSTM_EWC_w2b.json",
    pwd + "/2_experiment_results_LSTM_EWC_w2b.json",
    pwd + "/3_experiment_results_LSTM_EWC_w2b.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.1499958257443425
Number of entries: 48
Average domain training cost: 114.86003626267969
Number of entries: 47
Average FWT value for this file: -0.1654251437352345
Number of entries: 48
Average domain training cost: 101.07487378608494
Number of entries: 47
Average FWT value for this file: -0.17610869991819442
Number of entries: 48
Average domain training cost: 103.31445053189236
Overall average domain training cost: 106.41645352688566±6.040115510056604
Overall average FWT value: -0.16384322313259048±0.010719061028518552


In [36]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_EWC_toggle.json",
    pwd + "/2_experiment_results_LSTM_EWC_toggle.json",
    pwd + "/3_experiment_results_LSTM_EWC_toggle.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.12236628394833163
Number of entries: 48
Average domain training cost: 101.71760157844498
Number of entries: 47
Average FWT value for this file: -0.15133246107789267
Number of entries: 48
Average domain training cost: 103.71012121773674
Number of entries: 47
Average FWT value for this file: -0.133381032265256
Number of entries: 48
Average domain training cost: 109.10198567767414
Overall average domain training cost: 104.84323615795195±3.1193208910531873
Overall average FWT value: -0.1356932590971601±0.011937885046269268


## GR

In [37]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_GR_random.json",
    pwd + "/2_experiment_results_LSTM_GR_random.json",
    pwd + "/3_experiment_results_LSTM_GR_random.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.2475547746403671
Number of entries: 48
Average domain training cost: 35.78690964044654
Number of entries: 47
Average FWT value for this file: -0.2269012599671036
Number of entries: 48
Average domain training cost: 37.19064032591026
Number of entries: 47
Average FWT value for this file: -0.2577663125387628
Number of entries: 48
Average domain training cost: 36.34746421828944
Overall average domain training cost: 36.441671394882086±0.576929344847518
Overall average FWT value: -0.24407411571541116±0.012838720275705777


In [38]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_GR_b2w.json",
    pwd + "/2_experiment_results_LSTM_GR_b2w.json",
    pwd + "/3_experiment_results_LSTM_GR_b2w.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.1370497365640883
Number of entries: 48
Average domain training cost: 36.521891842208184
Number of entries: 47
Average FWT value for this file: -0.1283508454842342
Number of entries: 48
Average domain training cost: 37.31007162730384
Number of entries: 47
Average FWT value for this file: -0.13927442393549158
Number of entries: 48
Average domain training cost: 36.943272185167494
Overall average domain training cost: 36.92507855155984±0.3220301224380998
Overall average FWT value: -0.13489166866127136±0.004713391160659185


In [39]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_GR_w2b.json",
    pwd + "/2_experiment_results_LSTM_GR_w2b.json",
    pwd + "/3_experiment_results_LSTM_GR_w2b.json"
]


values = []
avg_fwt_list = []
avg_cost_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")

Number of entries: 47
Average FWT value for this file: -0.3705051869752491
Number of entries: 48
Average domain training cost: 35.71093283011578
Number of entries: 47
Average FWT value for this file: -0.3808197830019288
Number of entries: 48
Average domain training cost: 35.062259620041004
Number of entries: 47
Average FWT value for this file: -0.3864550046050656
Number of entries: 48
Average domain training cost: 37.23889601136519
Overall average domain training cost: 36.004029487173995±0.9124566593518181
Overall average FWT value: -0.37925999152741446±0.00660423514062893


In [ ]:
# List of files
files = [ pwd + "/1_experiment_results_LSTM_GR_toggle.json",
    pwd + "/2_experiment_results_LSTM_GR_toggle.json",
    pwd + "/3_experiment_results_LSTM_GR_toggle.json"
]


values = []
bwt_values = []
avg_fwt_list = []
avg_cost_list = []
avg_bwt_list = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        try:

            bwt_values.append(data["BWT_values"][46])
            fwt_dict = data.get("FWT_dict", {})
            values = list(fwt_dict.values())
            domain_training_cost = data.get("domain_training_cost", {})
            efficiency_values = [v[0] for v in domain_training_cost.values() if v]

        except (KeyError, IndexError) as e:
            print(f"Problem with {file}: {e}")
    if bwt_values:
        avg_bwt_list.append(np.mean(bwt_values))
        print(f"Number of entries: {len(bwt_values)}")
        print(f"Average BWT value for this set of files: {avg_bwt_list[-1]}")
    else:
        print("No BWT_dict found or empty.")
    
    if values:
        avg_fwt_list.append(np.mean(values))
        print(f"Number of entries: {len(values)}")
        print(f"Average FWT value for this file: {avg_fwt_list[-1]}")
    else:
        print("No FWT_dict found or empty.")
    
    if efficiency_values:
        avg_cost_list.append(np.mean(efficiency_values )) 
        print(f"Number of entries: {len(efficiency_values )}")
        print(f"Average domain training cost: {avg_cost_list[-1]}")
    else:
        print("No domain_training_cost found or empty.")

if avg_fwt_list:
    overall_avg_fwt = np.mean(avg_fwt_list)
    overall_std_fwt = np.std(avg_fwt_list)
    overall_avg_cost = np.mean(avg_cost_list)
    overall_std_cost = np.std(avg_cost_list)
    overall_avg_bwt = np.mean(avg_bwt_list)
    overall_std_bwt = np.std(avg_bwt_list)
    print(f"Overall average BWT value: {overall_avg_bwt}±{overall_std_bwt}")
    print(f"Overall average domain training cost: {overall_avg_cost}±{overall_std_cost}")
    print(f"Overall average FWT value: {overall_avg_fwt}±{overall_std_fwt}")
    
alpha = 0.3
beta = 0.4
gamma = 0.3
composite_score = (alpha * (overall_avg_fwt+1)/2) + (beta * (1- overall_avg_bwt)) + (gamma * overall_avg_cost)
print(f"Composite score: {composite_score}")

Number of entries: 1
Average BWT value for this set of files: -0.28772987271025996
Number of entries: 47
Average FWT value for this file: -0.30065718489814036
Number of entries: 48
Average domain training cost: 37.375932355833356
Number of entries: 2
Average BWT value for this set of files: -0.2990081492839731
Number of entries: 47
Average FWT value for this file: -0.2997019052680094
Number of entries: 48
Average domain training cost: 35.8324899506697
Number of entries: 3
Average BWT value for this set of files: -0.2992684725502792
Number of entries: 47
Average FWT value for this file: -0.2634257712312175
Number of entries: 48
Average domain training cost: 36.61016928045137
Overall average BWT value: -0.2953354981815041±0.005379039329184677
Overall average domain training cost: 36.606197195651475±0.6301139831319351
Overall average FWT value: -0.2879282871324558±0.017330283781303375
Composite score: 11.413614871828308
